In [4]:
from tensorflow import keras
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras_self_attention import SeqSelfAttention
from keras.layers.convolutional import Conv2D, MaxPooling2D


import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('D:\\Dissertation\\csi-dataset-20-4-5.pkl')

In [3]:
Y = df['Label'].values
X = df['Sample'].values
X = [[np.asarray(sample) for sample in i] for i in X]
X = np.asarray(X).astype('float32')

In [5]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
vectorized_y = np_utils.to_categorical(encoded_Y)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,vectorized_y, test_size=0.3, random_state=40)

In [7]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],X_test.shape[2],1))

X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)

In [8]:
X_train.shape

TensorShape([3311, 256, 180, 1])

In [9]:
Y_train.shape

TensorShape([3311, 5])

In [10]:
# def mish(x):
# 	return tf.keras.layers.Lambda(lambda x: x*tf.tanh(tf.log(1+tf.exp(x))), )(x)

def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x,inputs)
    return x

In [11]:
def csiTime_block(input_layer):
    layer_1 = Conv2D(10, (1), padding='same', activation='relu')(input_layer)
    layer_1 = Conv2D(10, (10), padding='same', activation='relu')(layer_1)

    layer_2 = Conv2D(10, (1), padding='same', activation='relu')(input_layer)
    layer_2 = Conv2D(10, (20), padding='same', activation='relu')(layer_2)

    layer_3 = Conv2D(10, (3), strides=(1), padding='same')(input_layer)
    layer_3 = Conv2D(10, (40), padding='same', activation='relu')(layer_3)

    mid_1 = keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)

    layer_4 = keras.layers.BatchNormalization()(mid_1)
    
    # layer_4 = SeqSelfAttention(attention_width=15,attention_activation='sigmoid')(layer_4)
    layer_4 = keras.layers.MaxPooling2D()(layer_4)
    # layer_4 = keras.layers.Flatten()(layer_4)
    output_layer = keras.layers.Dense(8, activation='relu')(layer_4)
    
    return output_layer

In [12]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape, dtype='float32')

    fir = csiTime_block(input_layer)
    sec = csiTime_block(fir)
    thi = csiTime_block(sec)
    fou = csiTime_block(thi)
    fif = csiTime_block(fou)
    six = csiTime_block(fif)

    gap = keras.layers.GlobalAveragePooling2D()(sec)
    # layer_4 = keras.layers.Flatten()(layer_4)
    output_layer = keras.layers.Dense(5, activation=mish)(gap) #Need to change to num of classes
    
    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 180, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 180, 10  20          ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 180, 10  20          ['input_1[0][0]']                
                                )                                                             

In [89]:
epochs = 25
batch_size = 256

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model2.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
history = model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/25


KeyboardInterrupt: 

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(X_test, Y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()